# Similarity Ensemble

In [1]:
%load_ext autoreload

from os import chdir
chdir('../')

from src.utils.loader import *
from scipy.sparse import *
import numpy as np
from src.utils.feature_weighting import *
from src.utils.matrix_utils import compute_cosine, top_k_filtering, yadistance
from src.utils.BaseRecommender import BaseRecommender


class ContentBasedFiltering(BaseRecommender):

    """
    Good conf: tag aggr 3,10; tfidf l1 norm over all matrix
    MAP@5  0.11772497678137457 with 10 shrinkage,
                                    100 k_filtering and other as before
    MAP@5  0.12039006297936491 urm weight 0.7
    MAP@5  0.12109109578826009 without playcount and duration

    Current best:
    CBF (album 1.0, artists 1.0, no duration/playcount)
        + URM 0.8
        + TOP-55 (TFIDF (tags 1.0))
        MAP@5 0.11897304011860126
        Public leaderboard: 0.09616


    """

    def __init__(self, shrinkage=10, k_filtering=100):
        # final matrix of predictions
        self.R_hat = None

        # for keeping reference between playlist and row index
        self.pl_id_list = []
        # for keeping reference between tracks and column index
        self.tr_id_list = []

        self.shrinkage = shrinkage
        self.k_filtering = k_filtering

    def fit(self, urm, target_playlist, target_tracks, dataset, topK_tag=55):
        """
        urm: user rating matrix
        target playlist is a list of playlist id
        target_tracks is a list of track id
        shrinkage: shrinkage factor for significance weighting
        S = ICM' ICM
        R = URM S
        In between eliminate useless row of URM and useless cols of S
        """
        # initialization

        self.pl_id_list = list(target_playlist)
        self.tr_id_list = list(target_tracks)
        self.dataset = dataset
        S = None
        urm = urm.tocsr()
        print("CBF started")
        # get ICM from dataset, assume it already cleaned
        icm = dataset.build_icm()

        # Build the tag matrix, apply TFIDF
        print("Build tags matrix and apply TFIDF...")
        icm_tag = dataset.build_tags_matrix()
        #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
        #print(tag_weight[tag_weight>5].shape)
        #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
        tags = applyTFIDF(icm_tag, norm='l1')
        #tags = icm_tag[tag_weight>5]
        # Before stacking tags with the rest of the ICM, we keep only
        # the top K tags for each item. This way we try to reduce the
        # natural noise added by such sparse features.
        tags = top_k_filtering(tags.transpose(), topK=topK_tag).transpose()
        #tags.data = tags.data / np.max(tags.data)
        #newvalue= (max'-min')/(max-min)*(value-max)+max'

        # User augmented UCM
        # print("Building User augmented ICM")
        # ucm = dataset.build_ucm()
        # ua_icm = user_augmented_icm(urm, ucm)
        # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

        # stack all
        icm = vstack([icm, tags, urm * 1], format='csr')
        # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

        S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                            for x in self.tr_id_list]],
                       icm,
                       k_filtering=self.k_filtering,
                       shrinkage=self.shrinkage,
                       n_threads=4,
                       chunksize=1000)
        s_norm = S.sum(axis=1)

        # Normalize S
        S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
        print("Similarity matrix ready!")

        # Keep only the target playlists in the URM
        urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                           for x in self.pl_id_list]]
        self.S = S.transpose()

        # Compute ratings
        R_hat = urm_cleaned.dot(S.transpose().tocsc()).tocsr()
        print("R_hat done")

        # Remove the entries in R_hat that are already present in the URM
        urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                      for x in self.tr_id_list]]
        R_hat[urm_cleaned.nonzero()] = 0
        R_hat.eliminate_zeros()

        print("Shape of final matrix: ", R_hat.shape)
        self.R_hat = R_hat

    def getW(self):
        """
        Returns the similary matrix with dimensions I x I
        S is IxT
        """
        return self.S.tocsr()

    def predict(self, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, self.R_hat.shape[0]):
            pl_id = self.pl_id_list[i]
            pl_row = self.R_hat.data[self.R_hat.indptr[i]:
                                     self.R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [self.R_hat.indices[self.R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [self.tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

    def getR_hat(self):
        return self.R_hat

    def get_model(self):
        """
        Returns the complete R_hat
        """
        return self.R_hat.copy()


def applyTFIDF(matrix, norm='l1'):
    from sklearn.feature_extraction.text import TfidfTransformer
    transformer = TfidfTransformer(norm=norm, use_idf=True,
                                   smooth_idf=True, sublinear_tf=False)
    tfidf = transformer.fit_transform(matrix.transpose())
    return tfidf.transpose()


def produceCsv():
    # export csv
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    cbf_exporter = ContentBasedFiltering()
    urm = dataset.build_train_matrix()
    tg_playlist = list(dataset.target_playlists.keys())
    tg_tracks = list(dataset.target_tracks.keys())
    # Train the model with the best shrinkage found in cross-validation
    cbf_exporter.fit(urm,
                     tg_playlist,
                     tg_tracks,
                     dataset)
    recs = cbf_exporter.predict()
    with open('submission_cbf.csv', mode='w', newline='') as out:
        fieldnames = ['playlist_id', 'track_ids']
        writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
        writer.writeheader()
        for k in tg_playlist:
            track_ids = ''
            for r in recs[k]:
                track_ids = track_ids + r + ' '
            writer.writerow({'playlist_id': k,
                             'track_ids': track_ids[:-1]})


def evaluateMap():
    from src.utils.evaluator import Evaluator
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    # seed = 0xcafebabe
    # print("Evaluating with initial seed: {}".format(seed))
    ev = Evaluator(seed=False)
    ev.cross_validation(5, dataset.train_final.copy())
    cbf = ContentBasedFiltering()
    for i in range(0, 5):
        urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
        cbf.fit(urm,
                list(tg_playlist),
                list(tg_tracks),
                dataset)
        recs = cbf.predict()
        ev.evaluate_fold(recs)

    map_at_five = ev.get_mean_map()
    print("MAP@5 ", map_at_five)


def crossValidation():
    from src.utils.evaluator import Evaluator
    pass

# Item Based Similarity

In [2]:
def ibs(urm, dataset):
    urm_sim = applyTFIDF(urm)
    #urm_sim = top_k_filtering(urm_sim.transpose()).transpose()
    S = compute_cosine(urm_sim.transpose()[[dataset.get_track_index_from_id(x)
                                    for x in tg_tracks]],
                       urm_sim,
                       k_filtering=100,
                       shrinkage=10,
                       n_threads=4,
                       chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# User Based Similarity

In [3]:
def ubs(urm, ucm, dataset):
    ucm = dataset.build_ucm()
    iucm = ucm.dot(urm)
    iucm = applyTFIDF(iucm)
    iucm = top_k_filtering(iucm.transpose(), 100).transpose()
    S = compute_cosine(iucm.transpose()[[dataset.get_track_index_from_id(x)
                                                for x in tg_tracks]],
                           iucm,
                           k_filtering=100,
                           shrinkage=10,
                           n_threads=4,
                           chunksize=250)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# Only Content Based

In [4]:
def icm_sim(icm, dataset):
    icm_tag = dataset.build_tags_matrix()
    #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
    #print(tag_weight[tag_weight>5].shape)
    #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
    tags = applyTFIDF(icm_tag, norm='l1')
    #tags = icm_tag[tag_weight>5]
    # Before stacking tags with the rest of the ICM, we keep only
    # the top K tags for each item. This way we try to reduce the
    # natural noise added by such sparse features.
    tags = top_k_filtering(tags.transpose(), topK=55).transpose()
    #tags.data = tags.data / np.max(tags.data)
    #newvalue= (max'-min')/(max-min)*(value-max)+max'

    # User augmented UCM
    # print("Building User augmented ICM")
    # ucm = dataset.build_ucm()
    # ua_icm = user_augmented_icm(urm, ucm)
    # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

    # stack all
    icm = vstack([icm, tags*0.4], format='csr')
    #icm = applyTFIDF(icm)
    #icm = top_k_filtering(icm.transpose(), topK=200).transpose()
    # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

    S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                        for x in tg_tracks]],
                   icm,
                   k_filtering=100,
                   shrinkage=10,
                   n_threads=4,
                   chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# Initialization

In [5]:
from src.utils.evaluator import Evaluator
from src.utils.loader import *
print("started")
print("hey")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())
urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
print("Done")

started
hey
File found, retrieving urm from it.
Load from file takes 0.78 seconds
Done


# Ensemble

### SLIM BPR

In [6]:
%autoreload 2

from src.ML.BPRSLIM import BPRSLIM
from src.utils.matrix_utils import applyTfIdf, top_k_filtering

import os
dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)

print('Building the ICM for SLIMBPR...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTfIdf(icm_tag)

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

Building the ICM for SLIMBPR...


In [7]:
slim = BPRSLIM(epochs=30,
               epochMultiplier=1.0,
               sgd_mode='rmsprop',
               learning_rate=5e-05,
               topK=300,
               urmSamplingChances=1 / 5,
               icmSamplingChances=4 / 5)

In [8]:
slim.set_evaluation_every(5, ev)
slim.fit(urm.tocsr(),
         icm.tocsr(),
         tg_playlist,
         tg_tracks,
         dataset)

S_bprslim = slim.getParameters()
S_bprslim = S_bprslim[:,[dataset.get_track_index_from_id(x) for x in tg_tracks]]

# keep only 100 elements
S_bprslim = top_k_filtering(S_bprslim.transpose(), 100).transpose()

# normalize
s_norm = S_bprslim.sum(axis=0)
s_norm[s_norm==0] = 1
print("S done")

S_bprslim = S_bprslim.multiply(csr_matrix(np.reciprocal(s_norm)))

Running fit process.
Processed 500000 ( 29.85% ) in 57.98 seconds. Sample per second: 8624
Processed 1000000 ( 59.70% ) in 25.25 seconds. Sample per second: 12157
Processed 1500000 ( 89.54% ) in 17.78 seconds. Sample per second: 15032
Processed 1675142 ( 100.00% ) in 6.83 seconds. Sample per second: 15828
Return S matrix to python caller...
Epoch 0 of 30 complete in 2.12 minutes
Processed 500000 ( 29.85% ) in 14.78 seconds. Sample per second: 33834
Processed 1000000 ( 59.70% ) in 14.29 seconds. Sample per second: 35343
Processed 1500000 ( 89.54% ) in 13.65 seconds. Sample per second: 36015
Processed 1675142 ( 100.00% ) in 5.61 seconds. Sample per second: 35940
Return S matrix to python caller...
Epoch 1 of 30 complete in 1.38 minutes
Processed 500000 ( 29.85% ) in 14.90 seconds. Sample per second: 33562
Processed 1000000 ( 59.70% ) in 13.84 seconds. Sample per second: 35914
Processed 1500000 ( 89.54% ) in 13.92 seconds. Sample per second: 36654
Processed 1675142 ( 100.00% ) in 5.47 sec

/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


MAP@5: 0.0937027821759783
Epoch 5 of 30 complete in 2.02 minutes
Processed 500000 ( 29.85% ) in 15.22 seconds. Sample per second: 32860
Processed 1000000 ( 59.70% ) in 15.84 seconds. Sample per second: 32428
Processed 1500000 ( 89.54% ) in 14.68 seconds. Sample per second: 33573
Processed 1675142 ( 100.00% ) in 5.47 seconds. Sample per second: 33860
Return S matrix to python caller...
Epoch 6 of 30 complete in 1.70 minutes
Processed 500000 ( 29.85% ) in 17.24 seconds. Sample per second: 28999
Processed 1000000 ( 59.70% ) in 15.96 seconds. Sample per second: 30338
Processed 1500000 ( 89.54% ) in 14.72 seconds. Sample per second: 32105
Processed 1675142 ( 100.00% ) in 5.57 seconds. Sample per second: 32483
Return S matrix to python caller...
Epoch 7 of 30 complete in 1.75 minutes
Processed 500000 ( 29.85% ) in 18.29 seconds. Sample per second: 27337
Processed 1000000 ( 59.70% ) in 16.33 seconds. Sample per second: 29128
Processed 1500000 ( 89.54% ) in 13.95 seconds. Sample per second: 31

Return S matrix to python caller...
Epoch 27 of 30 complete in 1.68 minutes
Processed 500000 ( 29.85% ) in 18.08 seconds. Sample per second: 27662
Processed 1000000 ( 59.70% ) in 17.14 seconds. Sample per second: 28461
Processed 1500000 ( 89.54% ) in 13.29 seconds. Sample per second: 31064
Processed 1675142 ( 100.00% ) in 5.03 seconds. Sample per second: 31586
Return S matrix to python caller...
Epoch 28 of 30 complete in 1.72 minutes
Processed 500000 ( 29.85% ) in 17.06 seconds. Sample per second: 29312
Processed 1000000 ( 59.70% ) in 15.85 seconds. Sample per second: 30446
Processed 1500000 ( 89.54% ) in 13.60 seconds. Sample per second: 32897
Processed 1675142 ( 100.00% ) in 4.92 seconds. Sample per second: 33554
Return S matrix to python caller...
Epoch 29 of 30 complete in 1.67 minutes
Fit completed in 52.87 minutes
S done


### Matrix Factorization

In [1]:
from src.CBF.CBF_MF import ContentBasedFiltering as CBF
from src.MF.MF_BPR.MF_BPR import MF_BPR

# Init ICM params
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
cbf = CBF()
    
cbf.fit(urm, tg_playlist,
        tg_tracks,
        dataset)

# get R_hat
R_hat_aug = cbf.getR_hat()
print(R_hat_aug.nnz)

mf = MF_BPR()

ModuleNotFoundError: No module named 'src'

In [ ]:
# MAP@5: 0.08256503053607782 with 500 factors after 10 epochs
# MAP@5: 0.08594586443489391 with 500 factors afetr 4 epochs no_components=500, epoch_multiplier=2, l_rate=1e-2
mf.fit(R_hat_aug, dataset, list(tg_playlist), list(tg_tracks), n_epochs=1, no_components=500, epoch_multiplier=6, l_rate=1e-2, use_icm=False)
recs = mf.predict_dot_custom(urm)
ev.evaluate_fold(recs)

# MAP@5: 0.09407901681369218 with neighborhood
# MAP@5: 0.09854105406016736 with neighborhood after 4 epochs
recs = mf.predict_knn_custom(urm)
ev.evaluate_fold(recs)

S_mf = mf.S

### Full Content Based Filtering

In [9]:
print("ok")
# Initiliaze Models
dataset.set_track_attr_weights_2(1.5, 1.6, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
cbf = ContentBasedFiltering()
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)
#recs = cbf.predict()
#ev.evaluate_fold(recs)
S_cbf_full = cbf.S

ok
CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 10965 ] Building cosine similarity matrix for [0, 1000)...
[ 10966 ] Building cosine similarity matrix for [4293, 5293)...
[ 10967 ] Building cosine similarity matrix for [8586, 9586)...
[ 10968 ] Building cosine similarity matrix for [12879, 13879)...
[ 10965 ] Building cosine similarity matrix for [1000, 2000)...
[ 10967 ] Building cosine similarity matrix for [9586, 10586)...
[ 10968 ] Building cosine similarity matrix for [13879, 14879)...
[ 10966 ] Building cosine similarity matrix for [5293, 6293)...
[ 10965 ] Building cosine similarity matrix for [2000, 3000)...
[ 10967 ] Building cosine similarity matrix for [10586, 11586)...
[ 10968 ] Building cosine similarity matrix for [14879, 15879)...
[ 10966 ] Building cosine similarity matrix for [6293, 7293)...
[ 10965 ] Building cosine similarity matrix for [3000, 4000)...
[ 10967 ] Building cosine similarity matrix for [11586, 12586)...
[ 10968 ] Building cosi

### URM Item based filtering

In [10]:
S_urm = ibs(urm, dataset)

Running 4 workers...
[ 11050 ] Building cosine similarity matrix for [0, 1000)...
[ 11051 ] Building cosine similarity matrix for [4293, 5293)...
[ 11052 ] Building cosine similarity matrix for [8586, 9586)...
[ 11053 ] Building cosine similarity matrix for [12879, 13879)...
[ 11050 ] Building cosine similarity matrix for [1000, 2000)...
[ 11051 ] Building cosine similarity matrix for [5293, 6293)...
[ 11052 ] Building cosine similarity matrix for [9586, 10586)...
[ 11053 ] Building cosine similarity matrix for [13879, 14879)...
[ 11050 ] Building cosine similarity matrix for [2000, 3000)...
[ 11051 ] Building cosine similarity matrix for [6293, 7293)...
[ 11052 ] Building cosine similarity matrix for [10586, 11586)...
[ 11053 ] Building cosine similarity matrix for [14879, 15879)...
[ 11050 ] Building cosine similarity matrix for [3000, 4000)...
[ 11052 ] Building cosine similarity matrix for [11586, 12586)...
[ 11051 ] Building cosine similarity matrix for [7293, 8293)...
[ 11053 ] B

### Only ICM Content based filtering

In [11]:
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

[ 11109 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 11110 ] Building cosine similarity matrix for [4293, 5293)...
[ 11111 ] Building cosine similarity matrix for [8586, 9586)...
[ 11112 ] Building cosine similarity matrix for [12879, 13879)...
[ 11109 ] Building cosine similarity matrix for [1000, 2000)...
[ 11112 ] Building cosine similarity matrix for [13879, 14879)...
[ 11111 ] Building cosine similarity matrix for [9586, 10586)...
[ 11110 ] Building cosine similarity matrix for [5293, 6293)...
[ 11111 ] Building cosine similarity matrix for [10586, 11586)...
[ 11109 ] Building cosine similarity matrix for [2000, 3000)...
[ 11110 ] Building cosine similarity matrix for [6293, 7293)...
[ 11112 ] Building cosine similarity matrix for [14879, 15879)...
[ 11111 ] Building cosine similarity matrix for [11586, 12586)...
[ 11109 ] Building cosine similarity matrix for [3000, 4000)...
[ 11110 ] Building cosine similarity matrix for [7293, 8293)...
[ 11112 ] B

### User Content Based Filtering

In [12]:
dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

Running 4 workers...
[ 11167 ] Building cosine similarity matrix for [0, 250)...
[ 11168 ] Building cosine similarity matrix for [4293, 4543)...
[ 11169 ] Building cosine similarity matrix for [8586, 8836)...
[ 11170 ] Building cosine similarity matrix for [12879, 13129)...
[ 11167 ] Building cosine similarity matrix for [250, 500)...
[ 11168 ] Building cosine similarity matrix for [4543, 4793)...
[ 11170 ] Building cosine similarity matrix for [13129, 13379)...
[ 11169 ] Building cosine similarity matrix for [8836, 9086)...
[ 11168 ] Building cosine similarity matrix for [4793, 5043)...
[ 11167 ] Building cosine similarity matrix for [500, 750)...
[ 11170 ] Building cosine similarity matrix for [13379, 13629)...
[ 11169 ] Building cosine similarity matrix for [9086, 9336)...
[ 11168 ] Building cosine similarity matrix for [5043, 5293)...
[ 11167 ] Building cosine similarity matrix for [750, 1000)...
[ 11170 ] Building cosine similarity matrix for [13629, 13879)...
[ 11169 ] Building c

In [13]:
def predict(R_hat, pl_id_list, tr_id_list, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, R_hat.shape[0]):
            pl_id = pl_id_list[i]
            pl_row = R_hat.data[R_hat.indptr[i]:
                                     R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [R_hat.indices[R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

# Bayesian opt

In [ ]:
def mixS(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_bprslim, S_user, urm, tg_playlist, evaluator
    print(params)

    S = S_icm * params[0] + S_urm * params[1] + S_cbf_full * params[2] + S_bprslim * params[3] + S_user * params[4]
    print("Similarity matrix ready!")

    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S= S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    print("Shape of final matrix: ", R_hat.shape)
    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return -map_at_five

In [ ]:
# BEST with ICM, URM, CBF_FULL, SLIMBPR
# x = [0.010884732743304261, 0.02719900735954462, 0.048684679013303325, 0.022415631857985566]
from skopt import forest_minimize
from skopt.space import Integer, Real
space = [Real(0.0, 1.0),  # ICM
         Real(0.0, 1.0),  # URM
         Real(0.0, 1.0),  # CBF_FULL
         Real(0.0, 1.0),  # SLIMBPR
         Real(0.0, 1.0)  # USER
        ]
x0 = [1, 0, 0, 0, 0]
x1 = [0, 1, 0, 0, 0]
x2 = [0, 0, 1, 0, 0]
x3 = [0, 0, 0, 1, 0]
x4 = [0, 0, 0, 0, 1]
x5 = [0.010884732743304261, 0.02719900735954462, 0.048684679013303325, 0.022415631857985566, 0.0]
x6 = [0.25550064193970895, 0.09295686755690992, 0.73164854390113454, 0.23952134524950303, 0.99898125828950768]
x0s = [x0, x1, x2, x3, x4, x5, x6]
res = forest_minimize(mixS, space, x0=x0s, verbose=True, n_random_starts=50, n_calls=1000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['ICM', 'URM', 'CBF_FULL', 'SLIMBPR', 'USER']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.10297444017190692
[0, 1, 0, 0, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.08815124783231548
[0, 0, 1, 0, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11615019226419378
[0, 0, 0, 1, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done


/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (4421, 17173)
MAP@5: 0.10771921887958982
[0, 0, 0, 0, 1]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.07657241951293055
[0.010884732743304261, 0.02719900735954462, 0.048684679013303325, 0.022415631857985566, 0.0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11779989444318799
[0.25550064193970895, 0.09295686755690992, 0.7316485439011345, 0.23952134524950303, 0.9989812582895077]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11835482168438524
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 223.5085
Function value obtained: -0.1184
Current minimum: -0.1184
Iteration No: 2 started. Evaluating function at provided point.
[0.67802525874511221, 0.79654307345303499, 0.6630520895471429, 0.62983476100845259, 0.12671902816203554]
Similarity matrix ready!
Similarity ma

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11612832692452693
Iteration No: 19 ended. Evaluation done at random point.
Time taken: 33.4980
Function value obtained: -0.1161
Current minimum: -0.1186
Iteration No: 20 started. Evaluating function at random point.
[0.53216171113190891, 0.47040046619338116, 0.49653372453040812, 0.64265433888422951, 0.64156168527120017]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11793334841287814
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 33.1396
Function value obtained: -0.1179
Current minimum: -0.1186
Iteration No: 21 started. Evaluating function at random point.
[0.77012116208386583, 0.94148651284006402, 0.37174200770593485, 0.90824253730676641, 0.10430993120678303]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.1162059865792053
Iteration No: 2

R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11796652341099316
Iteration No: 38 ended. Evaluation done at random point.
Time taken: 33.0810
Function value obtained: -0.1180
Current minimum: -0.1188
Iteration No: 39 started. Evaluating function at random point.
[0.53774813480567107, 0.25688880452130763, 0.93182250415051671, 0.86105062948646172, 0.78965408829333361]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11813767624217768
Iteration No: 39 ended. Evaluation done at random point.
Time taken: 32.2328
Function value obtained: -0.1181
Current minimum: -0.1188
Iteration No: 40 started. Evaluating function at random point.
[0.017934232780556284, 0.32791432104717783, 0.52213523695050823, 0.11094312141335852, 0.19839511749602529]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11632586895875757
Iteration No: 40 ended. Evaluation done at random point.
Time t

...

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11597074568348043
Iteration No: 608 ended. Search finished for the next optimal point.
Time taken: 30.4975
Function value obtained: -0.1160
Current minimum: -0.1188
Iteration No: 609 started. Searching for the next optimal point.
[0.031474188571852697, 0.18720597054414204, 0.025342669056973695, 0.076448147038381223, 0.17021112904039773]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11532081731131719
Iteration No: 609 ended. Search finished for the next optimal point.
Time taken: 30.3057
Function value obtained: -0.1153
Current minimum: -0.1188
Iteration No: 610 started. Searching for the next optimal point.
[0.01006886973978627, 0.27380999368100239, 0.26033266212259415, 0.072400427200620579, 0.81115876503304163]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11587423659805507
Iteration No: 627 ended. Search finished for the next optimal point.
Time taken: 30.3139
Function value obtained: -0.1159
Current minimum: -0.1188
Iteration No: 628 started. Searching for the next optimal point.
[0.080259138985501091, 0.26005305736218193, 0.22661385678998985, 0.12613275639123903, 0.99991331847771592]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11138053230792418
Iteration No: 628 ended. Search finished for the next optimal point.
Time taken: 30.3810
Function value obtained: -0.1114
Current minimum: -0.1188
Iteration No: 629 started. Searching for the next optimal point.
[0.63257333090266565, 0.00019384354402673948, 0.027730850323656638, 0.34053773223405265, 0.083524808753824745]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.10811128703913143
Iteration No: 646 ended. Search finished for the next optimal point.
Time taken: 30.4455
Function value obtained: -0.1081
Current minimum: -0.1188
Iteration No: 647 started. Searching for the next optimal point.
[0.012747830955194609, 0.2513556029974045, 0.43168752441723024, 0.018044669129121397, 0.30225058245606046]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11393425318555384
Iteration No: 647 ended. Search finished for the next optimal point.
Time taken: 30.3878
Function value obtained: -0.1139
Current minimum: -0.1188
Iteration No: 648 started. Searching for the next optimal point.
[0.096132802425939939, 0.38541877376964717, 0.026471787125182618, 0.032481877349846096, 0.70496743624323543]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11196486466108739
Iteration No: 665 ended. Search finished for the next optimal point.
Time taken: 30.3899
Function value obtained: -0.1120
Current minimum: -0.1188
Iteration No: 666 started. Searching for the next optimal point.
[0.46540311042689364, 0.017434307137001451, 0.81184895470941376, 0.30719491470312926, 0.0073145978916890914]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11543165196411077
Iteration No: 666 ended. Search finished for the next optimal point.
Time taken: 30.3515
Function value obtained: -0.1154
Current minimum: -0.1188
Iteration No: 667 started. Searching for the next optimal point.
[0.064743740180746037, 0.71873844109648277, 0.093892039868694016, 0.18407427567930171, 0.99991987367549029]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.11250320440322732
Iteration No: 684 ended. Search finished for the next optimal point.
Time taken: 32.6446
Function value obtained: -0.1125
Current minimum: -0.1188
Iteration No: 685 started. Searching for the next optimal point.
[0.14991790671903973, 0.53129595207312241, 0.0044270369159766796, 0.80802831442914358, 0.027586690245793417]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5: 0.1162286058961021
Iteration No: 685 ended. Search finished for the next optimal point.
Time taken: 30.6205
Function value obtained: -0.1162
Current minimum: -0.1188
Iteration No: 686 started. Searching for the next optimal point.
[0.084148364615069818, 0.087661907146843535, 0.0025856244592135096, 0.031446481850221371, 0.72609312698382045]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17173)
MAP@5:

# Final submission

In [6]:
%autoreload 2

from src.utils.loader import *
print("started")
print("hey")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
urm = dataset.build_train_matrix()
tg_playlist = list(dataset.target_playlists.keys())
tg_tracks = list(dataset.target_tracks.keys())

started
hey
File found, retrieving urm from it.
Load from file takes 0.51 seconds


In [7]:
from src.ML.BPRSLIM import BPRSLIM
from src.utils.matrix_utils import applyTfIdf, top_k_filtering

import os
dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)

print('Building the ICM for SLIMBPR...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTfIdf(icm_tag)

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

Building the ICM for SLIMBPR...


In [8]:
print("Training SLIM BPR...")
slim = BPRSLIM(epochs=30,
               epochMultiplier=1.0,
               sgd_mode='rmsprop',
               learning_rate=5e-05,
               topK=300,
               urmSamplingChances=1 / 5,
               icmSamplingChances=4 / 5)

# If present, load SLIM BPR parameters from disk
w_path = './data/slim_bpr_full_parameters.npz'
if os.path.isfile(w_path):
    W = load_sparse_matrix(w_path)
    slim.W = W.tocsr()
    S_bprslim = W.tocsr()
    print("BPR SLIM parameters loaded...")
else:
    slim.fit(urm.tocsr(),
             icm.tocsr(),
             tg_playlist,
             tg_tracks,
             dataset)
    S_bprslim = slim.getParameters()

print("Cleaning S_bprslim")
S_bprslim = S_bprslim[:,[dataset.get_track_index_from_id(x) for x in tg_tracks]]

print("Filtering...")
# keep only 100 elements
S_bprslim = top_k_filtering(S_bprslim.transpose(), 100).transpose()

print("Normalizing...")
# normalize
s_norm = S_bprslim.sum(axis=0)
s_norm[s_norm==0] = 1
print("S done")

S_bprslim = S_bprslim.multiply(csr_matrix(np.reciprocal(s_norm)))

Training SLIM BPR...
BPR SLIM parameters loaded...
Cleaning S_bprslim
Filtering...
Normalizing...
S done


In [16]:
# If not present, save SLIM BPR parameters to disk
w_path = './data/slim_bpr_full_parameters.npz'
if not os.path.isfile(w_path):
    save_sparse_matrix(w_path, slim.getParameters())

In [9]:
print("ok")
# Initiliaze Models
dataset.set_track_attr_weights_2(1.5, 1.6, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
cbf = ContentBasedFiltering()
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)

S_cbf_full = cbf.S

ok
CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 8562 ] Building cosine similarity matrix for [0, 1000)...
[ 8563 ] Building cosine similarity matrix for [8048, 9048)...
[ 8564 ] Building cosine similarity matrix for [16096, 17096)...
[ 8565 ] Building cosine similarity matrix for [24144, 25144)...
[ 8562 ] Building cosine similarity matrix for [1000, 2000)...
[ 8563 ] Building cosine similarity matrix for [9048, 10048)...
[ 8564 ] Building cosine similarity matrix for [17096, 18096)...
[ 8565 ] Building cosine similarity matrix for [25144, 26144)...
[ 8562 ] Building cosine similarity matrix for [2000, 3000)...
[ 8565 ] Building cosine similarity matrix for [26144, 27144)...
[ 8564 ] Building cosine similarity matrix for [18096, 19096)...
[ 8563 ] Building cosine similarity matrix for [10048, 11048)...
[ 8562 ] Building cosine similarity matrix for [3000, 4000)...
[ 8563 ] Building cosine similarity matrix for [11048, 12048)...
[ 8565 ] Building cosine simila

In [10]:
S_urm = ibs(urm, dataset)

Running 4 workers...
[ 8748 ] Building cosine similarity matrix for [0, 1000)...
[ 8749 ] Building cosine similarity matrix for [8048, 9048)...
[ 8750 ] Building cosine similarity matrix for [16096, 17096)...
[ 8751 ] Building cosine similarity matrix for [24144, 25144)...
[ 8748 ] Building cosine similarity matrix for [1000, 2000)...
[ 8749 ] Building cosine similarity matrix for [9048, 10048)...
[ 8750 ] Building cosine similarity matrix for [17096, 18096)...
[ 8748 ] Building cosine similarity matrix for [2000, 3000)...
[ 8751 ] Building cosine similarity matrix for [25144, 26144)...
[ 8750 ] Building cosine similarity matrix for [18096, 19096)...
[ 8749 ] Building cosine similarity matrix for [10048, 11048)...
[ 8748 ] Building cosine similarity matrix for [3000, 4000)...
[ 8751 ] Building cosine similarity matrix for [26144, 27144)...
[ 8750 ] Building cosine similarity matrix for [19096, 20096)...
[ 8749 ] Building cosine similarity matrix for [11048, 12048)...
[ 8748 ] Building 

In [11]:
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

[ 8906 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 8907 ] Building cosine similarity matrix for [8048, 9048)...
[ 8908 ] Building cosine similarity matrix for [16096, 17096)...
[ 8909 ] Building cosine similarity matrix for [24144, 25144)...
[ 8909 ] Building cosine similarity matrix for [25144, 26144)...
[ 8906 ] Building cosine similarity matrix for [1000, 2000)...
[ 8908 ] Building cosine similarity matrix for [17096, 18096)...
[ 8907 ] Building cosine similarity matrix for [9048, 10048)...
[ 8909 ] Building cosine similarity matrix for [26144, 27144)...
[ 8906 ] Building cosine similarity matrix for [2000, 3000)...
[ 8908 ] Building cosine similarity matrix for [18096, 19096)...
[ 8907 ] Building cosine similarity matrix for [10048, 11048)...
[ 8908 ] Building cosine similarity matrix for [19096, 20096)...
[ 8907 ] Building cosine similarity matrix for [11048, 12048)...
[ 8906 ] Building cosine similarity matrix for [3000, 4000)...
[ 8909 ] Building 

In [12]:
dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

Running 4 workers...
[ 9021 ] Building cosine similarity matrix for [0, 250)...
[ 9022 ] Building cosine similarity matrix for [8048, 8298)...
[ 9023 ] Building cosine similarity matrix for [16096, 16346)...
[ 9024 ] Building cosine similarity matrix for [24144, 24394)...
[ 9023 ] Building cosine similarity matrix for [16346, 16596)...
[ 9022 ] Building cosine similarity matrix for [8298, 8548)...
[ 9021 ] Building cosine similarity matrix for [250, 500)...
[ 9024 ] Building cosine similarity matrix for [24394, 24644)...
[ 9023 ] Building cosine similarity matrix for [16596, 16846)...
[ 9021 ] Building cosine similarity matrix for [500, 750)...
[ 9022 ] Building cosine similarity matrix for [8548, 8798)...
[ 9024 ] Building cosine similarity matrix for [24644, 24894)...
[ 9023 ] Building cosine similarity matrix for [16846, 17096)...
[ 9022 ] Building cosine similarity matrix for [8798, 9048)...
[ 9024 ] Building cosine similarity matrix for [24894, 25144)...
[ 9021 ] Building cosine s

[ 9023 ] Building cosine similarity matrix for [24096, 24144)...
[ 9023 ] Building cosine similarity matrix for [32192, 32195)...
[ 9024 ] Building cosine similarity matrix for [32144, 32192)...
[ 9022 ] Building cosine similarity matrix for [16048, 16096)...
[ 9021 ] Building cosine similarity matrix for [8000, 8048)...
S done
Similarity matrix ready!


In [13]:
# Best params 0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967]
# [0.24995989924853496, 0.35681721890423695, 0.12389358733625891, 0.78110404084094665]
# weights = [0.53, 0.86, 0.45, 0.85]
S = S_icm * 0.25550064193970895 + S_urm * 0.09295686755690992 + S_cbf_full * 0.73164854390113454 + S_bprslim * 0.2395213452495030 + S_user * 0.99898125828950768
# S = S_icm * 0.010884732743304261 + S_urm * 0.02719900735954462 + S_cbf_full * 0.048684679013303325 + S_bprslim * 0.022415631857985566
print("Similarity matrix ready!")

s_norm = S.sum(axis=0)
s_norm[s_norm==0] = 1
# Normalize S
S = S.multiply(csr_matrix(np.reciprocal(s_norm)))

# Keep only the target playlists in the URM
urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                   for x in tg_playlist]]

# Compute ratings
R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
print("R_hat done")

# Remove the entries in R_hat that are already present in the URM
urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                              for x in tg_tracks]]
R_hat[urm_cleaned.nonzero()] = 0
R_hat.eliminate_zeros()

print("Shape of final matrix: ", R_hat.shape)
recs = predict(R_hat, list(tg_playlist), list(tg_tracks))

Similarity matrix ready!
R_hat done
Shape of final matrix:  (10000, 32195)


In [14]:
with open('submission_ensemble.csv', mode='w', newline='') as out:
    fieldnames = ['playlist_id', 'track_ids']
    writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
    writer.writeheader()
    for k in tg_playlist:
        track_ids = ''
        for r in recs[k]:
            track_ids = track_ids + r + ' '
        writer.writerow({'playlist_id': k,
                         'track_ids': track_ids[:-1]})